In [1]:
import pandas as pd

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#### Get Names from data sources
- Add all the LLM parts to this file and run?
- compile into one file

In [5]:
names = []
with open('westlaw_names.txt', 'r', encoding='utf-8') as f:
    for line in f:
        names.append(line.strip().capitalize())

with open('incident_names.txt', 'r', encoding='utf-8') as f:
    for line in f:
        names.append(line.strip().capitalize())

kyc = pd.read_csv('../data/raw/kyc.csv')

In [6]:
kyc

,name,gender,occupation,age,tenure,cust_id,label
0,JENNIFER WELLS,female,Architect,45.0,13.0,CUST82758793,0
1,ANTHONY ADAMS,male,Musician,52.0,8.0,CUST69248708,0
2,DENISE LEWIS,female,Jewelry Dealer,43.0,11.0,CUST67222818,0
3,STEPHEN FIGUEROA,male,Optometrist,35.0,17.0,CUST33995820,0
4,KYLE EDWARDS,male,Real Estate Broker,39.0,21.0,CUST76401392,1
...,...,...,...,...,...,...,...
195784,THOMAS YOUNG,male,Software Developer,46.0,3.0,CUST23014082,0
195785,CASEY JONES,male,Miner,31.0,10.0,CUST17691251,0
195786,NICOLE-CÉCILE LEBLANC,female,Unknown,21.0,3.0,CUST26444112,0
195787,CATHERINE ARMSTRONG,female,Antiques Dealer,58.0,14.0,CUST96567835,0


In [13]:


# Normalize data (to lowercase in this example)
internet_names = [name.lower() for name in names]
kyc_names = [name.lower() for name in kyc.name]

# Function to find best match
def find_best_match(name, ref_db, scorer):
    return process.extractOne(name, ref_db, scorer=fuzz.token_sort_ratio)

# Finding matches
matches = {name: find_best_match(name, kyc_names, fuzz.ratio) for name in internet_names}
matches_sort_ratio = {name: find_best_match(name, kyc_names, fuzz.token_sort_ratio) for name in internet_names}
matches_partial_ratio = {name: find_best_match(name, kyc_names, fuzz.partial_ratio) for name in internet_names}
matches_set_ratio = {name: find_best_match(name, kyc_names, fuzz.token_set_ratio) for name in internet_names}


KeyboardInterrupt: 

In [16]:
# Print results
for name, match in matches.items():
    if match[1] == 100:
        print(f'{name}: {match}')

for name, match in matches_sort_ratio.items():
    if match[1] > 90:
        print(f'{name}: {match}')

for name, match in matches_partial_ratio.items():
    if match[1] > 90:
        print(f'{name}: {match}')

james lewis: ('james lewis', 100)
richard austin.: ('richard austin', 100)
gilles allain: ('gilles allain', 100)
aaron jones: ('aaron jones', 100)
charlotte jones: ('charlotte jones', 100)
olivia terrance: ('olivia terrance', 100)
stéphane therrien: ('stéphane therrien', 100)
brian miller: ('brian miller', 100)
logan gregory: ('gregory logan', 100)
vanessa rondeau: ('vanessa rondeau', 100)
timothy lewis: ('timothy lewis', 100)
lisa peterson: ('lisa peterson', 100)
frank johnson: ('frank johnson', 100)
gregory logan: ('gregory logan', 100)
ryan hicks: ('ryan hicks', 100)
zhou hong xia: ('zhou hong xia', 100)
michael bryant: ('michael bryant', 100)
anthony nguyen: ('anthony nguyen', 100)
joshua harvey: ('joshua harvey', 100)
carlos rodriguez: ('carlos rodriguez', 100)
chang xiong: ('xiong chang', 100)
robert brewer: ('robert brewer', 100)
greg anderson: ('greg anderson', 100)
heather navarro: ('heather navarro', 100)
jacob lane: ('jacob lane', 100)
richard parker: ('richard parker', 100)

In [22]:
# save the dictionaries as parquet files using dataframes
matches_df = pd.DataFrame(columns=['name', 'match', 'score', 'ratio'])
matches_df['name'] = matches.keys()
matches_df['match'] = [el[0] for el in matches.values()]
matches_df['ratio'] = 'fuzz.ratio'
matches_df['score'] = [el[1] for el in matches.values()]

matches_sort_ratio_df = pd.DataFrame(columns=['name', 'match', 'score', 'ratio'])
matches_sort_ratio_df['name'] = matches_sort_ratio.keys()
matches_sort_ratio_df['match'] = [el[0] for el in matches_sort_ratio.values()]
matches_sort_ratio_df['ratio'] = 'fuzz.token_sort_ratio'
matches_sort_ratio_df['score'] = [el[1] for el in matches_sort_ratio.values()]

matches_partial_ratio_df = pd.DataFrame(columns=['name', 'match', 'score', 'ratio'])
matches_partial_ratio_df['name'] = matches_partial_ratio.keys()
matches_partial_ratio_df['match'] = [el[0] for el in matches_partial_ratio.values()]
matches_partial_ratio_df['ratio'] = 'fuzz.partial_ratio'
matches_partial_ratio_df['score'] = [el[1] for el in matches_partial_ratio.values()]

matches_df = pd.concat([matches_df, matches_sort_ratio_df, matches_partial_ratio_df])
matches_df

,name,match,score,ratio
0,andrew stanley paul randy larry chickite curti...,dr.tristan-michel blanchette,44,fuzz.ratio
1,john a. hofer,john hooper,78,fuzz.ratio
2,paul francis smallboy,francis malone,69,fuzz.ratio
3,irvine scalplock,camille boivin,60,fuzz.ratio
4,bear paw pawn ltd.,bernard pépin,62,fuzz.ratio
...,...,...,...,...
1245,josé moisés chávez quetz,monique choi,61,fuzz.partial_ratio
1246,sr.,sara de,44,fuzz.partial_ratio
1247,noel quintana,joe quinn,73,fuzz.partial_ratio
1248,the profepa,thérèse croteau,67,fuzz.partial_ratio


In [23]:
matches_df.to_parquet('matches.parquet')